3) In dieser Aufgabe soll nun ein kleines Neuronales Netz aus fünf Neuronen zur Erkennung eines Schachbrettmusters auf einem 2x2 Feld programmiert werden. Als Eingabe für das 2x2 Feld nutzen wir einen Vektor mit 4 Einträgen, wobei eine 1 ein schwarzes Feld und eine 0 ein weißes Feld darstellt.
Zu Beginn müssen wir die Klassen HHSimulationOdeint, HHModel und HHParameters aus Aufgabe 2 importieren. Wie bereits gezeigt, ist die odeint Methode die genaueste und schnellste Methode zum Berechnen der GDG und wird fortan verwendet.
Weiter enthält der Code in den folgenden Zeilen eine Methode zum Zeichnen des Schachbretts, die für ein vereinfachtes Verständnis sorgen soll.

In [1]:
import nbimporter
import numpy as np
import matplotlib.pyplot as plt

from Aufgabe_2 import HHSimulationOdeint, HHModel, HHParameters

# Methode zum Zeichnen des Schachbretts
def zeichne_schachbrett(vektor):
    
    # Wirft Fehler, wenn nicht 1 oder 0, da sonst Graustufen erlaubt wären oder Länge des Vektor ungleich 4
    if len(vektor) != 4 or any(bit not in [0, 1] for bit in vektor):
        raise ValueError("Der Vektor muss genau vier Zahlen enthalten, die 0 oder 1 sein können.")

    # Erstellt ein 2x2-Array 
    schachbrett = np.array(vektor).reshape(2, 2)
    
    # Vektor in String für Title der Grafik
   
    vector_str = ', '.join(map(str, vektor))
    
    # Erstellt das Schachbrett
    plt.imshow(schachbrett, cmap='gray_r', vmin=0, vmax=1)
    plt.xticks([])  # Entferne x-Achsenmarkierungen
    plt.yticks([])  # Entferne y-Achsenmarkierungen
    plt.title(f'Schachbrett, das als richtig erkannt werden soll')
    plt.show()

good_pattern = [1, 0, 0, 1]

# Zeichne das Schachbrett
zeichne_schachbrett(good_pattern) 

NameError: name 'eingangsvektor' is not defined

In [ ]:
params = HHParameters()
hh_model = HHModel(params)

Wir geben zur Implementierung der Trainingsdaten verschiedene Vektoren vor, die die unterschiedlichen möglichen Muster auf dem 2x2 Brett darstellen, in possible_patters vor. Dabei ist zu beachten, dass jedes Brett immer aus zwei weißen und zwei schwarzen Feldern besteht. good_pattern enthält den Vektor, den wir als korrekt erkennen wollen. Im gegebenen Fall ist dies das Brett, welches oben links und unten rechts ein schwarzes Feld aufweist.
Da das Neuronale Netz aus 5 Neuronen, also vier Input-Neuronen (je eins für jedes Feld auf dem Brett) und einem Output-Neuron (Muster vorhanden/Muster nicht vorhanden) bestehen soll, ist es nur möglich eine der beiden Diagonalen als richtiges Muster zu erkennen. Die optimalen Wichtungsfaktoren (diese sollen Werte zwischen 0 und 1 annehmen) zur Erkennung des Musters sind 1 für die gewünschten schwarzen Felder, also oben links und unten rechts und 0 auf den anderen beiden Feldern. 

Da ein guter Trainingsdatensatz etwa zur Hälfte aus dem richtigen Muster und zur anderen Hälfte aus falschen Mustern bestehen sollte, lassen wir den Trainingsdatensatz zu 40% aus dem good_pattern und zu 60% aus einem zufällig geshuffleten Vektor aus zwei Einsen und zwei Nullen bestehen. Der Trainingsdatensatz enthält 25 Muster.

In [ ]:
possible_patterns = [[1, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1], [0, 1, 1, 0], [0, 1, 0, 1], [0, 0, 1, 1]]
good_pattern = [1, 0, 0, 1]

training_data_length = 25
training_data = np.zeros((training_data_length, 4), dtype=int)

# updates the global training_data variable with new patterns
def make_new_training_data():
    for i in range(training_data_length):
        pattern = np.array([1, 1, 0, 0])
        
        # extra 40% chance of using the good pattern, so that the network can learn to recognize the good pattern
        if np.random.rand() < 0.4:
            pattern = good_pattern
        else: 
            np.random.shuffle(pattern)

        training_data[i] = pattern

make_new_training_data()

Nachdem der Satz für die Trainingsdaten initialisert wurde, kann nun die Klasse für das Neuronale Netz geschrieben werden. 
Die Wichtigunsfaktoren werden in einem Vektor Länge 4 dargestellt (Annahme für optimalen Vektor der Wichtungsfaktoren: [1, 0, 0, 1]) und per Skalarprodukt mit dem Vektor, der das Muster darstellt, verrechnet. 
Wenn man die die Wichtungsfaktoren  als Skalarprodukt mit dem Vektor für das als richtig zu erkennende Muster nimmt, erhalten wir 2 als Ergebniss, während ein Vektor, der nicht dem gesuchten Muster entspricht, eine 1 oder 0 als Ergbniss ausgeben würde.  
Dieses Ergbenis wird mit dem extern anliegenden Strom und dem Faktor 6 multipliziert und als ein sekündiger Inputstromimpuls für das HHModel genommen. So ist sichergestellt, dass nur bei einem richtigen Muster ein Ausschlag in der Spannung des HHModels entsteht. Der maximale Wert der Spannung U wird durch eine Sigmoidfunktion transformiert, um eine Entscheidung zu treffen.

In [ ]:
class NN_5_Neurons:

    # weights is a vector of length 4
    def __init__(self, weights):
        self.weights = np.array(weights)

    @staticmethod
    def getInputCurrent(weightedSum):
        def I_ext(t):
            return  6 * weightedSum if t < 1 else -5
        return I_ext

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def is14DiagonalBlack(self, pattern):
        weightedSum = np.dot(self.weights, pattern)
        I_ext = self.getInputCurrent(weightedSum)

        _, U, _, _, _ = HHSimulationOdeint(hh_model, time=10, dt=0.01, I_ext=I_ext).run()

        return self.sigmoid(max(U) / 10)



3.c) Zum Test sollen alle Wichtungsfaktoren auf 1 gesetzt werden (entspricht dem Vektor [1, 1, 1, 1]) und das Ergebnis mit den von uns als gut angenommenen Wichtungsfaktoren [1, 0, 0, 1] verglichen werden:

In [ ]:
# good weights to recognize the pattern 'good_pattern' 
good_weights = np.copy(good_pattern)

nn_ones = NN_5_Neurons(weights=[1, 1, 1, 1])
results_ones = [nn_ones.is14DiagonalBlack(pattern) for pattern in possible_patterns]

print(f"results for NN with 1 as all weights: {results_ones}")

nn_good = NN_5_Neurons(good_weights)
results_good = [nn_good.is14DiagonalBlack(pattern) for pattern in possible_patterns]

print(f"results for NN with good weights: {results_good}")

print(f"perfect results: {[1 if pattern == good_pattern else 0 for pattern in possible_patterns]}")

Für ein als richtig erkanntes Muster soll dabei einen Wert nahe 1 ausgebene werden, während ein als falsch erkanntes Muster einem Wert gegen 0 entspricht.
Es ist zu erkennen, dass wenn alle Wichtungsfaktoren auf 1 gesetzt sind, jedes Muster als "richtig" erkannt werden würde auch wenn es eigentlich ein falsches Muster ist. Dies entspricht den Erwartungen, da das Ergebnis des Skalarproduktest bei einem Vektor für die Wichtungsfaktoren von [1, 1, 1, 1] immer 2 ergibt.
Schaut man allerdings auf die Ergebnisse für die für gut angenommenen Wichtungsfaktoren, wird jedes Ergebnis mit einer Abweichung von maximal 0,0025 von dem gewünschten Ergebnis als richtig erkannt wird.

2.d)
Der Algorithmus zum maschinellen Lernen soll geschrieben werden. Dies ist eine einfache Implementation des Konzeptes der Backpropergation.
Dafür werden zuerst die Wichtungsfaktoren zufällig zwischen 0 und 1 gewählt.

Die Wichtungsfaktoren und der Loss werden in zwei Arrays während des Trainings zwischengespeichert.
Es wird ein zufälliges Muster vorgegeben und das Neuronale Netz errechnet ein Ergebnis zwischen 0 und 1. 
Der Loss stellt dabei die Abweichung des Ergebnisses vom zu erwartenden, richtigen Ergebnis (1 oder 0) für das entsprechende Pattern dar. 
Der Vektor für die Wichtungsfaktoren wird dann während des Trainings nach jedem Muster um den Loss multipliziert mit dem Vektor, der das Muster angibt, addiert. Es werden also immer nur die Wichtungsfaktoren für ein schwarzes Feld angepasst.
Bei einem positiven Loss (falsches Pattern wird als richtig erkannt) werden die entsprechenden Gewichtungsfaktoren also um dem den Loss erhöht. Sollte ein negativer Loss (richtiges Ergebnis wird als falsch erkannt) berrechnet worden sein, werden die Gewichtungsfaktoren entsprechend verringert. 
Dabei ist zu beachten, dass niemals die Gewichte 2 und 3 erhöht werden können. Es kann allerdings passieren, dass die Gewichtungsfaktoren für das erste und vierte Neuron verringert werden. Wenn beispielsweise der Vektor [1, 1, 0, 0] fälschlicherweise als richtig erkannt wird, werden dementsprechend Wichtungsfaktor eins und zwei mit dem berechneten Loss multipliziert und um den Betrag verringert. 
So wird dafür gesorgt, dass die Wichtungsfaktoren bei einer falschen Vorhersage solange angepasst werden bis das NN die Ergebnisse genau vorhersagt. 

Für das Training wird 10 mal mit unterschiedlichen Start-Wichtungsfaktoren gestartet und durchläuft je 25 Iterationen.

In [ ]:
import matplotlib.pyplot as plt


# loss_for_single_pattern can be positive and negative, corresponding to undershooting and overshooting the correct result,
# this will tell if the weights have to be increased or decreased in training
def loss_for_single_pattern(weights, pattern):
    nn = NN_5_Neurons(weights)
    return np.array_equal(pattern, good_pattern) - nn.is14DiagonalBlack(pattern)

# loss_total > 0 
def loss_total(weights):
    return sum(abs(np.array([loss_for_single_pattern(weights, pattern) for pattern in possible_patterns])))


def training():
    weights = np.random.rand(4)

    # we record the total loss and the weights after each training step:
    losses_during_training = np.array([0.] * training_data_length)
    weights_during_training = np.array([weights] * training_data_length)

    for i, pattern in enumerate(training_data):
        loss = loss_for_single_pattern(weights, pattern)

        losses_during_training[i] = loss_total(weights)
        weights_during_training[i] = weights

        # the weights that contributed to the loss are the ones that got 1 as input
        # those weights get decreased or increased depending on loss < 0 or loss > 0
        # the change in those weights is proportional to the absolute value of the loss 
        weights += loss * pattern
        weights = np.clip(weights, 0, 1)

    return losses_during_training, weights_during_training


biggest_final_loss = 0

for i in range(10):
    losses, weights = training()

    biggest_final_loss = max(biggest_final_loss, losses[-1])

    # we make new random training data to demonstrate robustness to changes in the training data set
    make_new_training_data()

    # Plot der Entwicklung des losses und der weights
    plt.figure(figsize=(12, 8))

    # Weights
    plt.subplot(2, 1, 1)
    plt.title(f"Training number {i + 1}:\nStarting weights: {list(np.round(weights[0], 3))}\nFinal loss: {losses[-1]}")

    plt.plot(weights[:, 0], label="w1")
    plt.plot(weights[:, 1], label="w2")
    plt.plot(weights[:, 2], label="w3")
    plt.plot(weights[:, 3], label="w4")

    plt.ylabel("Weights")
    plt.legend()

    # Loss
    plt.subplot(2, 1, 2)
    plt.plot(losses, label="Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()

print(f"biggest loss after training: {biggest_final_loss}")

Es zeigt sich, dass nach dem Training von dem Neuronalen Netz mit maximalen Abweichung von 0,035 ein richtiges von einem falschen Muster unterschieden können.
